# Combining your model with a model without outlier

In [1]:
import numpy as np
import pandas as pd
import time
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import log_loss
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
import gc

from sklearn.calibration import CalibratedClassifierCV

import plotly.plotly as py
import plotly.graph_objs as go

from sklearn.metrics import f1_score,precision_recall_curve,roc_curve, recall_score,precision_score
from sklearn.metrics import confusion_matrix, roc_auc_score

%matplotlib inline
import random
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.max_rows = 999
pd.options.display.max_columns  = 999

In [2]:
from hyperopt import hp
from hyperopt.pyll.stochastic import sample
from hyperopt import tpe
from hyperopt import Trials
from hyperopt import fmin

import json
import pickle

import csv
from hyperopt import STATUS_OK
from timeit import default_timer as timer

In [3]:
print(np.__version__)

1.16.2


In [4]:
%%time
# Path = '../input/elo-preproc-3/'
Path = '../input/ltfs-fin-model/'

train_df = pd.read_csv(Path+'train_preproc.csv',index_col=0)
# test_df = pd.read_csv(Path +'test_preproc.csv',index_col=0)

CPU times: user 2.42 s, sys: 272 ms, total: 2.69 s
Wall time: 2.7 s


## filtering out outliers

In [5]:
targetcol = 'loan_default'
target = train_df[targetcol]

## training model

In [6]:
enc_splits = 5

In [7]:
# def getenc(mask_without_outlier=None):
#     tr_encs = []
#     val_encs = []
#     test_encs = []
#     Path='../input/elo-output/'
# #     Path='../input/elo-target-encoding-100-splits/'

#     for i in range(0,enc_splits):
#         cur_tr_enc = pd.read_hdf(Path+'train_targetenc_feats'+str(i)+'.hdf')
#         cur_val_enc = pd.read_hdf(Path+'val_targetenc_feats'+str(i)+'.hdf')

#         if mask_without_outlier is not None:
#             cur_tr_enc = cur_tr_enc[mask_without_outlier]
#             cur_val_enc = cur_val_enc[mask_without_outlier]

#         tr_encs += [cur_tr_enc]
#         val_encs +=[ cur_val_enc]

#         test_encs += [pd.read_hdf(Path+'test_targetenc_feats'+str(i)+'.hdf')]
#         print('read complete for:',i)
        
#     return tr_encs,val_encs,test_encs

In [8]:
# def lgb_fit(regression,param,tr,y_tr,val,y_val,cur_feval):
#     if regression:
#         model = lgb.LGBMRegressor(**param)
#     else:
#         model = lgb.LGBMClassifier(**param)
        
#     model.fit(tr,y_tr,eval_set=[(val, y_val)],
#                     early_stopping_rounds=200,
#                     verbose=1000,
#                     eval_metric=cur_feval,
#              )
#     return model
# def lgb_predict(model,n_estimators,test):
#     preds= model.predict(test,num_iteration = getbestiteration(model,n_estimators))
#     return preds
# # def xgb_fit(regression,param,tr,y_tr,val,y_val,cur_feval):
# #     num_round= param['n_estimators']
# #     trn_data = xgb.DMatrix(data=tr, label=y_tr)
# #     val_data = xgb.DMatrix(data=val, label=y_val)
# #     watchlist = [(trn_data, 'train'), (val_data, 'valid')]

# #     model = xgb.train(param, trn_data, num_round, watchlist, 
# #                       early_stopping_rounds=200, verbose_eval=100)

# #     return model

# def xgb_fit(regression,param,tr,y_tr,val,y_val,cur_feval):
#     if regression:
#         model = xgb.XGBRegressor(**param)
 
#     else:
#         model = xgb.XGBClassifier(**param)

#     model.fit(tr,y_tr,eval_set=[(val, y_val)],
#                     early_stopping_rounds=200,
#                     verbose=50,
#                     eval_metric=cur_feval,
#              )
#     return model
# def xgb_predict(model,n_estimators,test):
#     preds= model.predict(test)
# #     preds = model.predict(xgb.DMatrix(test), ntree_limit=model.best_ntree_limit)
#     return preds
# def getbestiteration(model,n_estimators):
#     if hasattr(model, 'best_iteration'):
#         if model.best_iteration is None:
#             return n_estimators
#         else:
#             return model.best_iteration
#     elif hasattr(model, 'best_iteration_'):
#         if model.best_iteration_ is None:
#             return n_estimators
#         else:
#             return model.best_iteration_
    
#     return -1

# def xgb_getbestscore(model,cur_feval):
#     if isinstance(model,xgb.XGBRegressor) :
# #         print('model best_score object:',model.get_booster().best_score)
#         score= model.get_booster().best_score
#     elif isinstance(model,xgb.XGBClassifier):
#         print('model best_score object:',model.get_booster().best_score)
#         score= model.get_booster().best_score
       
#     return score

# def lgb_getbestscore(model,cur_feval):
#     if  isinstance(model,lgb.LGBMRegressor) :
#         score= model.best_score_['valid_0'][param['metric']]
#     else: #isinstance(model,lgb.LGBMClassifier) :
#         if (cur_feval is None):
#             score= model.best_score_['valid_0'][param['metric']]
#         else:
#             score= model.best_score_['valid_0']['f1_score']
#     return score
# # def xgb_getbestscore(model,cur_feval):
# #     return model.best_score

In [9]:
import time
def runlgb(train,target,param,cur_features,fold_to_start=None, fold_to_stop=None):

   
    oof = np.zeros(len(train))
#     predictions = np.zeros(len(test))
    start = time.time()
    valid_scores =[]; iterations=[]
    fold_importance_df = pd.DataFrame()
    

    folds = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=4590)
    indices = folds.split(train.values, target.values)
        
    for fold_, (trn_idx, val_idx) in enumerate(indices):
        if (fold_to_stop is not None):
            if (fold_ >=fold_to_stop):
                break
                
        if (fold_to_start is not None):
            if (fold_ < fold_to_start):
                continue
                
        print()
        print("fold n°{}".format(fold_))

        tr = train.iloc[trn_idx]
        val = train.iloc[val_idx]
        y_val = target.iloc[val_idx]
        y_tr = target.iloc[trn_idx]
        
        trn_data = lgb.Dataset(tr[cur_features], label=y_tr)#,, categorical_feature=categorical_feats)
        val_data = lgb.Dataset(val[cur_features], label=y_val)#,, categorical_feature=categorical_feats)
        
        num_round = param['n_estimators']
        print('param:',param)
        clf = lgb.train(param, trn_data, num_round, valid_sets = [val_data], verbose_eval=1000, 
                        early_stopping_rounds = 300)

        oof[val_idx] = clf.predict(val[cur_features], num_iteration=clf.best_iteration)

        fold_importance_df["feature"] = cur_features
        if fold_==0:
            fold_importance_df["importance"] =0
        fold_importance_df["importance"] += clf.feature_importance() / n_splits
        valid_scores+=[clf.best_score['valid_0'][param['metric']]]
        iterations +=[clf.best_iteration]
#         predictions += clf.predict(test[cur_features], num_iteration=clf.best_iteration) / folds.n_splits

    print('valid scores:',valid_scores)
    cv_score = roc_auc_score(target, oof)
    print("CV score: {:<8.5f}".format(cv_score))

    return fold_importance_df,oof,cv_score,valid_scores,iterations

In [10]:
# # tr_encs, val_encs,test_encs = getenc(mask_without_outlier=mask_without_outlier)
# tr_encs, val_encs,test_encs = getenc()
# print(tr_encs[0].shape)
# print(val_encs[0].shape)
# print(test_encs[0].shape)

In [11]:
# sel_enc_cols =['trans_merged_targetenc_merchant_id_mean']

In [12]:
exclude_cols =['Date.of.Birth','Employment.Type','DisbursalDate',
               'PERFORM_CNS.SCORE.DESCRIPTION','AVERAGE.ACCT.AGE','CREDIT.HISTORY.LENGTH',
               'MobileNo_Avl_Flag','disbursal_year','disbursal_day','disbursal_dayofweek',
               'date_of_birth', 'disbursal_date',
               'UniqueID',targetcol]
features = [c for c in train_df.columns if c not in exclude_cols]
print(features)

['Aadhar_flag', 'Current_pincode_ID', 'DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS', 'Driving_flag', 'Employee_code_ID', 'NEW.ACCTS.IN.LAST.SIX.MONTHS', 'NO.OF_INQUIRIES', 'PAN_flag', 'PERFORM_CNS.SCORE', 'PRI.ACTIVE.ACCTS', 'PRI.CURRENT.BALANCE', 'PRI.DISBURSED.AMOUNT', 'PRI.NO.OF.ACCTS', 'PRI.OVERDUE.ACCTS', 'PRI.SANCTIONED.AMOUNT', 'PRIMARY.INSTAL.AMT', 'Passport_flag', 'SEC.ACTIVE.ACCTS', 'SEC.CURRENT.BALANCE', 'SEC.DISBURSED.AMOUNT', 'SEC.INSTAL.AMT', 'SEC.NO.OF.ACCTS', 'SEC.OVERDUE.ACCTS', 'SEC.SANCTIONED.AMOUNT', 'State_ID', 'VoterID_flag', 'asset_cost', 'branch_id', 'disbursed_amount', 'ltv', 'manufacturer_id', 'supplier_id', 'age', 'disbursal_month', 'PERFORM_CNS.SCORE.CATEGORY', 'Employment.Type.Category', 'AVERAGE.ACCT.AGE_MONTHS', 'CREDIT.HISTORY.LENGTH_MONTHS']


In [13]:
n_splits =5
fold_to_start = 0
fold_to_stop = 1

In [14]:
import datetime
def computescore(hyperparameters,train,features,target):
    
    hyperparameters['learning_rate'] = 0.01
    hyperparameters['boosting'] = 'gbdt'
    hyperparameters['bagging_seed'] = 2018
    hyperparameters['bagging_freq'] = 2
    hyperparameters['min_data_in_bin'] = 100
    hyperparameters['n_estimators'] = 10000
    hyperparameters['objective'] = "binary"
    hyperparameters['metric'] = "auc"
    hyperparameters['random_state'] = 2333
    hyperparameters['max_depth'] = 15
    hyperparameters['scale_pos_weight'] = 1  
    
    fold_importance_df,oof,cv_score,valid_scores,iterations\
        = runlgb(train,target,hyperparameters,features,
                 fold_to_start=fold_to_start,fold_to_stop=fold_to_stop)
    

    return cv_score, valid_scores, iterations

In [15]:
param = {'colsample_bytree': 0.7196484570790651,
   'min_child_samples': 235,
   'num_leaves': 36,
   'reg_alpha': 0.6474702076362333,
   'reg_lambda': 0.021458900986429996,
   'subsample': 0.8873887256306612,
   'subsample_for_bin': 110000,
   'learning_rate': 0.01,
   'boosting': 'gbdt',
   'bagging_seed': 2018,
   'bagging_freq': 2,
   'min_data_in_bin': 100,
   'n_estimators': 10000,
   'objective': 'binary',
   'metric': 'auc',
   'random_state': 2333,
   'max_depth': 15,
   'scale_pos_weight': 1}

In [16]:
# import time
# def runlgb_normal(ispermutefeats,train,test,param,cur_features,score_function=None):

#     overall_sel_feats =[]
#     overall_imp_df = pd.DataFrame()
#     overall_imp_df['feature']= np.array(cur_features)
#     overall_imp_df['overall_score_mean'] =0 
#     overall_imp_df['overall_score_max'] =-9999 
#     overall_imp_df['overall_score_min'] =9999 
    
#     oof = np.zeros(len(train))
#     predictions = np.zeros(len(test))
#     start = time.time()
#     valid_scores =[]
#     fold_importance_df = pd.DataFrame()
    

#     folds = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=4590)
#     indices = folds.split(train.values, target.values)
        
#     for fold_, (trn_idx, val_idx) in enumerate(indices):
#         print()
#         print("fold n°{}".format(fold_))

#         tr = train.iloc[trn_idx]
#         val = train.iloc[val_idx]
#         y_val = target.iloc[val_idx]
#         y_tr = target.iloc[trn_idx]
        
# #         val_index_ser = pd.Series(np.array(val.index))
# #         print('val shape:',val.shape)
# #         print('val index head:',val_index_ser.head(20))
# #         print('val index tail:',val_index_ser.tail(20))
        
#         trn_data = lgb.Dataset(tr[cur_features], label=y_tr)#,, categorical_feature=categorical_feats)
#         val_data = lgb.Dataset(val[cur_features], label=y_val)#,, categorical_feature=categorical_feats)
        
#         clf = lgb.train(param, trn_data, num_round, valid_sets = [val_data], verbose_eval=500, 
#                         early_stopping_rounds = 300)

#         #Prediction based on current fold selected features
#         if ispermutefeats:
            
#             selected_features, importance_df = permutation_feature_selection(clf, val[cur_features], 
#                                                                              y_val,score_function,
#                                                                              rep=4,max_delta_score=max_delta_score)
#             overall_sel_feats += [selected_features]
#             print(selected_features)

# #             print('overal imp shape:{0} importance_df shape:{1}'.format(overall_imp_df.shape,importance_df.shape))
            
#             overall_imp_df['fold_'+str(fold_)+'score_mean'] = importance_df['delta_score_mean']
#             overall_imp_df['fold_'+str(fold_)+'score_max'] = importance_df['delta_score_max']
#             overall_imp_df['fold_'+str(fold_)+'score_min'] = importance_df['delta_score_min']
#         else:
#             oof[val_idx] = clf.predict(val[cur_features], num_iteration=clf.best_iteration)

#             fold_importance_df["feature"] = cur_features
#             if fold_==0:
#                 fold_importance_df["importance"] =0
#             fold_importance_df["importance"] += clf.feature_importance() / n_splits
#             valid_scores+=[clf.best_score['valid_0'][param['metric']]]
#             predictions += clf.predict(test[cur_features], num_iteration=clf.best_iteration) / folds.n_splits

#     if ispermutefeats:
#         fold_mean_cols = [col for col in overall_imp_df.columns if ('score_mean' in col) and ('fold_' in col) ]
#         fold_max_cols = [col for col in overall_imp_df.columns if ('score_max' in col) and ('fold_' in col) ]
#         fold_min_cols = [col for col in overall_imp_df.columns if ('score_min' in col) and ('fold_' in col) ]
#         overall_imp_df['overall_score_mean'] = overall_imp_df[fold_mean_cols].mean(axis=1)
#         overall_imp_df['overall_score_max'] = overall_imp_df[fold_max_cols].max(axis=1)
#         overall_imp_df['overall_score_min'] = overall_imp_df[fold_min_cols].min(axis=1)
#     else:
#         print('valid scores:',valid_scores)
#         print("CV score: {:<8.5f}".format(roc_auc_score(target, oof)))

#     return fold_importance_df,predictions,oof,overall_imp_df,overall_sel_feats

In [17]:
# num_round = param['n_estimators']
# fold_importance_df,predictions_2,oof_2,overall_imp_df,overall_sel_feats = \
#         runlgb_normal(False,train_df,test_df,param,features,score_function=None)

In [18]:
def objective(hyperparameters):
    """Objective function for Gradient Boosting Machine Hyperparameter Optimization.
       Writes a new line to `outfile` on every iteration"""
    
    # Keep track of evals
    global ITERATION
    
    ITERATION += 1
    print('ITERATION:',ITERATION)
    
    print('hyper params')
    print('subsample: {0} num_leaves: {1} subsample_for_bin: {2}'.format(\
            hyperparameters['subsample'],hyperparameters['num_leaves'],hyperparameters['subsample_for_bin']))
    print('min_child_samples: {0} reg_alpha: {1} reg_lambda: {2} colsample_bytree: {3}'.format(\
            hyperparameters['min_child_samples'],hyperparameters['reg_alpha'],
            hyperparameters['reg_lambda'],hyperparameters['colsample_bytree']))
    
#     # Using early stopping to find number of trees trained
#     if 'n_estimators' in hyperparameters:
#         del hyperparameters['n_estimators']
    
#     # Retrieve the subsample
#     subsample = hyperparameters['boosting_type'].get('subsample', 1.0)
    
#     # Extract the boosting type and subsample to top level keys
#     hyperparameters['boosting_type'] = hyperparameters['boosting_type']['boosting_type']
#     hyperparameters['subsample'] = subsample
    
    # Make sure parameters that need to be integers are integers
    for parameter_name in ['num_leaves', 'subsample_for_bin', 'min_child_samples']:
        hyperparameters[parameter_name] = int(hyperparameters[parameter_name])

    start = timer()
    

    
    # Perform n_folds cross validation
    best_score,val_scores,n_estimators = computescore(hyperparameters,train_df,
                                                      features,target)
    
#     cv_results = lgb.cv(hyperparameters, train_set, num_boost_round = 10000, nfold = N_FOLDS, 
#                         early_stopping_rounds = 100, metrics = 'auc', seed = 50)

    run_time = timer() - start
    
#     Add the number of estimators to the hyperparameters
    hyperparameters['n_estimators'] = n_estimators

    # Loss must be minimized
    loss = 1 - best_score
    
    # Write to the csv file ('a' means append)
    of_connection = open(OUT_FILE, 'a')
    writer = csv.writer(of_connection)
    writer.writerow([loss, hyperparameters, ITERATION, run_time, best_score,val_scores])
    of_connection.close()

    # Dictionary with information for evaluation
    return {'loss': loss, 'hyperparameters': hyperparameters, 'iteration': ITERATION,
            'train_time': run_time, 'status': STATUS_OK}


In [19]:
# Define the search space
space = {
#     'boosting_type': hp.choice('boosting_type', 
#                                             [{'boosting_type': 'gbdt', 'subsample': hp.uniform('gdbt_subsample', 0.5, 1)}, 
#                                              {'boosting_type': 'dart', 'subsample': hp.uniform('dart_subsample', 0.5, 1)},
#                                              {'boosting_type': 'goss', 'subsample': 1.0}]),
    'subsample': hp.uniform('subsample', 0.6, 1),
    'num_leaves': hp.quniform('num_leaves', 20, 80, 1),
#     'bagging_freq' : hp.quniform('bagging_freq', 2, 25, 1),
#     'learning_rate': hp.loguniform('learning_rate', np.log(0.02), np.log(0.1)),
    'subsample_for_bin': hp.quniform('subsample_for_bin', 80000, 160000, 10000),
    'min_child_samples': hp.quniform('min_child_samples',30, 500, 5),
    'reg_alpha': hp.uniform('reg_alpha', 0.0, 1.0),
    'reg_lambda': hp.uniform('reg_lambda', 0.0, 1.0),
    'colsample_bytree': hp.uniform('colsample_by_tree', 0.6, 1.0),
#     'is_unbalance': hp.choice('is_unbalance', [True, False]),
}

In [20]:
tpe_algorithm = tpe.suggest
# Record results
trials = Trials()
# import json
# import pickle

Path = "../input/ltfs-fin-bayesian/"

# # with open(Path+'trials.json', 'r') as f:
trials = pickle.load(open(Path+"trials_cls.pickle", "rb"))

In [21]:
# Create a file and open a connection
OUT_FILE = 'bayes_elo_cls.csv'
of_connection = open(OUT_FILE, 'w')
writer = csv.writer(of_connection)

ITERATION = 0

# Write column names
headers = ['loss', 'hyperparameters', 'iteration', 'runtime', 'score','valid_scores']
writer.writerow(headers)
of_connection.close()

In [22]:
param ={'metric':'auc'}


In [23]:
%%time
# Global variable
global  ITERATION

ITERATION = 0
MAX_EVALS = 200+100
# Run optimization
best = fmin(fn = objective, space = space, algo = tpe.suggest, trials = trials,
            max_evals = MAX_EVALS)

best

ITERATION:
1
hyper params
subsample: 0.988456640881867 num_leaves: 64.0 subsample_for_bin: 140000.0
min_child_samples: 115.0 reg_alpha: 0.46711636846944976 reg_lambda: 0.8012130503992826 colsample_bytree: 0.6450913216384198
fold n°0
param:
{'colsample_bytree': 0.6450913216384198, 'min_child_samples': 115, 'num_leaves': 64, 'reg_alpha': 0.46711636846944976, 'reg_lambda': 0.8012130503992826, 'subsample': 0.988456640881867, 'subsample_for_bin': 140000, 'learning_rate': 0.01, 'boosting': 'gbdt', 'bagging_seed': 2018, 'bagging_freq': 2, 'min_data_in_bin': 100, 'n_estimators': 10000, 'objective': 'binary', 'metric': 'auc', 'random_state': 2333, 'max_depth': 15, 'scale_pos_weight': 1}
  0%|          | 0/100 [00:00<?, ?it/s, best loss: ?]

/opt/conda/lib/python3.6/site-packages/lightgbm/engine.py:118: UserWarning:

Found `n_estimators` in params. Will use it instead of argument




Training until validation scores don't improve for 300 rounds.
[1000]	valid_0's auc: 0.674277
[2000]	valid_0's auc: 0.675868
[3000]	valid_0's auc: 0.676083
Early stopping, best iteration is:
[2765]	valid_0's auc: 0.676229
valid scores:
[0.6762289505087588]
CV score: 0.50706 
ITERATION:
2
hyper params
subsample: 0.9694401211538677 num_leaves: 76.0 subsample_for_bin: 110000.0
min_child_samples: 45.0 reg_alpha: 0.28807085806697474 reg_lambda: 0.8368969422238387 colsample_bytree: 0.6277635388357228
fold n°0
param:
{'colsample_bytree': 0.6277635388357228, 'min_child_samples': 45, 'num_leaves': 76, 'reg_alpha': 0.28807085806697474, 'reg_lambda': 0.8368969422238387, 'subsample': 0.9694401211538677, 'subsample_for_bin': 110000, 'learning_rate': 0.01, 'boosting': 'gbdt', 'bagging_seed': 2018, 'bagging_freq': 2, 'min_data_in_bin': 100, 'n_estimators': 10000, 'objective': 'binary', 'metric': 'auc', 'random_state': 2333, 'max_depth': 15, 'scale_pos_weight': 1}
  1%|          | 1/100 [01:53<3:05:25

/opt/conda/lib/python3.6/site-packages/lightgbm/engine.py:118: UserWarning:

Found `n_estimators` in params. Will use it instead of argument




Training until validation scores don't improve for 300 rounds.
[1000]	valid_0's auc: 0.674336
[2000]	valid_0's auc: 0.676063
Early stopping, best iteration is:
[2443]	valid_0's auc: 0.676212
valid scores:
[0.6762123357300509]
CV score: 0.50706 
ITERATION:
3
hyper params
subsample: 0.9476250469715116 num_leaves: 23.0 subsample_for_bin: 100000.0
min_child_samples: 100.0 reg_alpha: 0.5450666256007439 reg_lambda: 0.9502078300837423 colsample_bytree: 0.847677012224715
fold n°0
param:
{'colsample_bytree': 0.847677012224715, 'min_child_samples': 100, 'num_leaves': 23, 'reg_alpha': 0.5450666256007439, 'reg_lambda': 0.9502078300837423, 'subsample': 0.9476250469715116, 'subsample_for_bin': 100000, 'learning_rate': 0.01, 'boosting': 'gbdt', 'bagging_seed': 2018, 'bagging_freq': 2, 'min_data_in_bin': 100, 'n_estimators': 10000, 'objective': 'binary', 'metric': 'auc', 'random_state': 2333, 'max_depth': 15, 'scale_pos_weight': 1}
  2%|▏         | 2/100 [03:33<2:57:36, 108.74s/it, best loss: 0.492906

...

Training until validation scores don't improve for 300 rounds.
[1000]	valid_0's auc: 0.674818
[2000]	valid_0's auc: 0.676567
Early stopping, best iteration is:
[2109]	valid_0's auc: 0.676634
valid scores:
[0.6766341341065009]
CV score: 0.50707 
ITERATION:
99
hyper params
subsample: 0.8882938634360983 num_leaves: 70.0 subsample_for_bin: 80000.0
min_child_samples: 230.0 reg_alpha: 0.21598429493362925 reg_lambda: 0.9076216970958334 colsample_bytree: 0.6099278789026381
fold n°0
param:
{'colsample_bytree': 0.6099278789026381, 'min_child_samples': 230, 'num_leaves': 70, 'reg_alpha': 0.21598429493362925, 'reg_lambda': 0.9076216970958334, 'subsample': 0.8882938634360983, 'subsample_for_bin': 80000, 'learning_rate': 0.01, 'boosting': 'gbdt', 'bagging_seed': 2018, 'bagging_freq': 2, 'min_data_in_bin': 100, 'n_estimators': 10000, 'objective': 'binary', 'metric': 'auc', 'random_state': 2333, 'max_depth': 15, 'scale_pos_weight': 1}
 98%|█████████▊| 98/100 [2:32:12<02:58, 89.10s/it, best loss: 0.492

/opt/conda/lib/python3.6/site-packages/lightgbm/engine.py:118: UserWarning:

Found `n_estimators` in params. Will use it instead of argument




Training until validation scores don't improve for 300 rounds.
[1000]	valid_0's auc: 0.675153
[2000]	valid_0's auc: 0.67624
Early stopping, best iteration is:
[1715]	valid_0's auc: 0.676473
valid scores:
[0.6764728404702253]
CV score: 0.50707 
ITERATION:
100
hyper params
subsample: 0.9067310847255 num_leaves: 78.0 subsample_for_bin: 150000.0
min_child_samples: 160.0 reg_alpha: 0.3174977999459879 reg_lambda: 0.7459424525684414 colsample_bytree: 0.6317729043020681
fold n°0
param:
{'colsample_bytree': 0.6317729043020681, 'min_child_samples': 160, 'num_leaves': 78, 'reg_alpha': 0.3174977999459879, 'reg_lambda': 0.7459424525684414, 'subsample': 0.9067310847255, 'subsample_for_bin': 150000, 'learning_rate': 0.01, 'boosting': 'gbdt', 'bagging_seed': 2018, 'bagging_freq': 2, 'min_data_in_bin': 100, 'n_estimators': 10000, 'objective': 'binary', 'metric': 'auc', 'random_state': 2333, 'max_depth': 15, 'scale_pos_weight': 1}
 99%|█████████▉| 99/100 [2:33:32<01:26, 86.46s/it, best loss: 0.492906711

/opt/conda/lib/python3.6/site-packages/lightgbm/engine.py:118: UserWarning:

Found `n_estimators` in params. Will use it instead of argument




Training until validation scores don't improve for 300 rounds.
[1000]	valid_0's auc: 0.6748
[2000]	valid_0's auc: 0.676289
Early stopping, best iteration is:
[1701]	valid_0's auc: 0.67637
valid scores:
[0.6763703810898282]
CV score: 0.50706 
100%|██████████| 100/100 [2:34:54<00:00, 85.21s/it, best loss: 0.4929067119721624]
CPU times: user 9h 52min 51s, sys: 14min 20s, total: 10h 7min 11s
Wall time: 2h 34min 54s


In [24]:
# Sort the trials with lowest loss first
trials_dict = sorted(trials.results, key = lambda x: x['loss'])
trials_dict[:1]

[{'loss': 0.4929067119721624,
  'hyperparameters': {'colsample_bytree': 0.6070263484848221,
   'min_child_samples': 120,
   'num_leaves': 65,
   'reg_alpha': 0.4086395992151187,
   'reg_lambda': 0.8723592642214507,
   'subsample': 0.9764323431114542,
   'subsample_for_bin': 160000,
   'learning_rate': 0.01,
   'boosting': 'gbdt',
   'bagging_seed': 2018,
   'bagging_freq': 2,
   'min_data_in_bin': 100,
   'n_estimators': [2377],
   'objective': 'binary',
   'metric': 'auc',
   'random_state': 2333,
   'max_depth': 15,
   'scale_pos_weight': 1},
  'iteration': 78,
  'train_time': 97.4711585569894,
  'status': 'ok'}]

In [25]:
results = pd.read_csv(OUT_FILE)

In [26]:
import json

pickle.dump(trials, open("trials_cls.pickle", "wb"))
# Save the trial results
# with open('trials.json', 'w') as f:
#     f.write(json.dumps(trials_dict))

In [27]:
import ast

def evaluate(results, name,train_df,y_reg):
    """Evaluate model on test data using hyperparameters in results
       Return dataframe of hyperparameters"""
    
    new_results = results.copy()
    # String to dictionary
    new_results['hyperparameters'] = new_results['hyperparameters'].map(ast.literal_eval)
    
    # Sort with best values on top
    new_results = new_results.sort_values('score', ascending = False).reset_index(drop = True)
    
    # Print out cross validation high score
    print('The highest cross validation score from {} was {:.5f} found on iteration {}.'.format(name, new_results.loc[0, 'score'], new_results.loc[0, 'iteration']))
    
    # Use best hyperparameters to create a model
    hyperparameters = new_results.loc[0, 'hyperparameters']
    
    hyp_df = pd.DataFrame(columns = list(new_results.loc[0, 'hyperparameters'].keys()))

    # Iterate through each set of hyperparameters that were evaluated
    for i, hyp in enumerate(new_results['hyperparameters']):
        hyp_df = hyp_df.append(pd.DataFrame(hyp, index = [0]), 
                               ignore_index = True)
        
    # Put the iteration and score in the hyperparameter dataframe
    hyp_df['iteration'] = new_results['iteration']
    hyp_df['score'] = new_results['score']
    hyp_df['valid_scores'] = new_results['valid_scores']
    
    return hyp_df

In [28]:
bayes_results = evaluate(results, 'Bayesian', train_df,target)
bayes_results.to_csv('bayes_elo_cls_tabular.csv')
bayes_results

The highest cross validation score from Bayesian was 0.50709 found on iteration 73.


,colsample_bytree,min_child_samples,num_leaves,reg_alpha,reg_lambda,subsample,subsample_for_bin,learning_rate,boosting,bagging_seed,bagging_freq,min_data_in_bin,n_estimators,objective,metric,random_state,max_depth,scale_pos_weight,iteration,score,valid_scores
0,0.641797,60,76,0.994722,0.940123,0.981634,160000,0.01,gbdt,2018,2,100,2526,binary,auc,2333,15,1,73,0.507089,[0.677042755384619]
1,0.607925,115,75,0.300099,0.949717,0.983533,160000,0.01,gbdt,2018,2,100,2175,binary,auc,2333,15,1,71,0.507080,[0.6768204493478739]
2,0.621359,150,73,0.307295,0.793069,0.953702,160000,0.01,gbdt,2018,2,100,2054,binary,auc,2333,15,1,64,0.507080,[0.6768030742478867]
3,0.626301,140,76,0.252713,0.903210,0.990027,160000,0.01,gbdt,2018,2,100,1979,binary,auc,2333,15,1,66,0.507079,[0.6767994268703607]
4,0.657893,45,67,0.990661,0.999136,0.945740,160000,0.01,gbdt,2018,2,100,2486,binary,auc,2333,15,1,78,0.507078,[0.6767729035630919]
5,0.643208,230,61,0.304983,0.779514,0.945783,160000,0.01,gbdt,2018,2,100,2240,binary,auc,2333,15,1,24,0.507078,[0.6767535532512398]
6,0.632566,130,65,0.482405,0.970312,0.965524,160000,0.01,gbdt,2018,2,100,2261,binary,auc,2333,15,1,61,0.507075,[0.6766967740274316]
7,0.600761,200,63,0.514043,0.861794,0.980713,160000,0.01,gbdt,2018,2,100,2193,binary,auc,2333,15,1,60,0.507074,[0.6766657442607642]
8,0.622421,130,72,0.339125,0.979915,0.981154,90000,0.01,gbdt,2018,2,100,2109,binary,auc,2333,15,1,98,0.507073,[0.6766341341065009]
9,0.600749,30,74,0.945721,0.955755,0.931900,160000,0.01,gbdt,2018,2,100,2188,binary,auc,2333,15,1,94,0.507070,[0.676570176475737]
